In [ ]:
import sqlite3
from flask import Flask, render_template, request
import numpy as np
import urllib.request
from bs4 import BeautifulSoup

#플라스크 객체 생성
app=Flask(__name__)

conn = sqlite3.connect("music.db")
cursor = conn.cursor()

In [ ]:
@app.route('/home')
def home():
    return render_template('home.html')

In [ ]:
@app.route('/melon')
def melon():
    return render_template('크롤링.html')

@app.route('/melon1', methods=['POST'])
def melon1():

    web_url = request.form['id']

    #### tablename에 table이 있는 경우 그냥 데이터 삭제후 다시 웹 크롤링
    def chart_balad_save():
        url = web_url # 멜론 발라드 차트 url 
        hdr = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.61 Safari/537.36"}
        req = urllib.request.Request(url, headers=hdr)
        html = urllib.request.urlopen(req).read()
        soup = BeautifulSoup(html, 'html.parser')

        conn = sqlite3.connect("music.db")
        cursor = conn.cursor()


        
        lst50 = soup.select('.lst50, .lst100')

        genre = []

        paging = soup.find('div', class_ = 'calendar_prid').get_text().replace("\n","")
        genre.append([paging])

        cursor.execute("DELETE FROM {}".format(genre[0]))

        melonList = []
        for i in lst50:
            ranking = (i.select_one("td > div > span.rank").text+"위") # 멜론 사이트에서 발라드 순위 가져와서 temp에 추가
            title = (i.select_one("td > div > div > div.ellipsis.rank01").a.text) # 멜론 사이트에서 노래 제목 가져오서 temp에 추가
            singer = (i.select_one('td > div > div > div.ellipsis.rank02').a.text) # 멜론 사이트에서 가수 이름 가져와서 temp에 추가
            album = (i.select_one('td > div > div > div.ellipsis.rank03').a.text) # 멜론 사이트에서 앨범 이름 가져와서 temp에 주가
            url_title = "https://www.youtube.com/results?search_query={}".format((i.select_one("td > div > div > div.ellipsis.rank01").a.text.replace(" ","+")))
            url_singer = "https://www.youtube.com/results?search_query={}".format((i.select_one('td > div > div > div.ellipsis.rank02').a.text.replace(" ","+")))
            melonList.append([ranking, title, singer, album]) # 멜론리스트에 temp에 있는 것들 추가
            
            inser='insert into {} (ranking, title, singer, album, youtube_title, youtube_singer) values(?,?,?,?,?,?)'.format(genre[0])
            cursor.execute(inser,(ranking, title, singer, album, url_title, url_singer))
        conn.commit()


    #### teblename에 없는 table일 경우 table을 생성하고 데이터 저장
    def chart_balad_save1():
        url = web_url # 멜론 발라드 차트 url 
        hdr = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.61 Safari/537.36"}
        req = urllib.request.Request(url, headers=hdr)
        html = urllib.request.urlopen(req).read()
        soup = BeautifulSoup(html, 'html.parser')

        conn = sqlite3.connect("music.db")
        cursor = conn.cursor()

        lst50 = soup.select('.lst50, .lst100')

        genre = []

        paging = soup.find('div', class_ = 'calendar_prid').get_text().replace("\n","")
        genre.append([paging])


        inser='insert into tablename (name) values(?)'
        cursor.execute(inser,(genre[0]))
        conn.commit()


        insert = 'create table {} (ranking REAL, title TEXT, singer TEXT, album TEXT, youtube_title TEXT, youtube_singer TEXT)'.format(genre[0])
        cursor.execute(insert)
        conn.commit()

        melonList = []
        for i in lst50:
            ranking = (i.select_one("td > div > span.rank").text+"위") # 멜론 사이트에서 발라드 순위 가져와서 temp에 추가
            title = (i.select_one("td > div > div > div.ellipsis.rank01").a.text) # 멜론 사이트에서 노래 제목 가져오서 temp에 추가
            singer = (i.select_one('td > div > div > div.ellipsis.rank02').a.text) # 멜론 사이트에서 가수 이름 가져와서 temp에 추가
            album = (i.select_one('td > div > div > div.ellipsis.rank03').a.text) # 멜론 사이트에서 앨범 이름 가져와서 temp에 주가
            url_title = "https://www.youtube.com/results?search_query={}".format((i.select_one("td > div > div > div.ellipsis.rank01").a.text.replace(" ","+")))
            url_singer = "https://www.youtube.com/results?search_query={}".format((i.select_one('td > div > div > div.ellipsis.rank02').a.text.replace(" ","+")))
            melonList.append([ranking, title, singer, album]) # 멜론리스트에 temp에 있는 것들 추가
            
            inser='insert into {} (ranking, title, singer, album, youtube_title, youtube_singer) values(?,?,?,?,?,?)'.format(genre[0])
            cursor.execute(inser,(ranking, title, singer, album, url_title, url_singer))
        conn.commit()


    url = web_url # 멜론 발라드 차트 url 
    hdr = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.61 Safari/537.36"}
    req = urllib.request.Request(url, headers=hdr)
    html = urllib.request.urlopen(req).read()
    soup = BeautifulSoup(html, 'html.parser')


    genre = []

    paging = soup.find('div', class_ = 'calendar_prid').get_text().replace("\n","")
    genre.append([paging])


    conn=sqlite3.connect('music.db')
    conn.row_factory=sqlite3.Row
    cursor=conn.cursor()

    cursor.execute('select*from tablename where name =? ',(paging,))
    row = cursor.fetchone()
    conn.close()

    if row!=None:
        if paging == row['name']:
            chart_balad_save()
    else:
        chart_balad_save1()


    conn=sqlite3.connect('music.db')
    conn.row_factory=sqlite3.Row
    cursor=conn.cursor()
    #계정 id 레코드 읽기
    #cursor.execute('select name from tablename')
    cursor.execute('select*from tablename')
    rows=cursor.fetchall()

    ins = []

    for row in rows:
        ins.append(row['name'])

    conn.close()
    return render_template('melon1.html', values = ins)

In [ ]:
@app.route('/melon11')
def melon11():
    conn=sqlite3.connect('music.db')
    conn.row_factory=sqlite3.Row
    cursor=conn.cursor()
    #계정 id 레코드 읽기
    #cursor.execute('select name from tablename')
    cursor.execute('select*from tablename')
    rows=cursor.fetchall()

    ins = []

    for row in rows:
        ins.append(row['name'])

    conn.close()
    return render_template('melon1.html', values = ins)

In [ ]:
@app.route('/melon2', methods=['POST'])
def melon2():

    ch = request.form['tablename']
    table = []
    table.append([ch])

    conn = sqlite3.connect("music.db")
    # conn.row_factory=sqlite3.Row
    cur = conn.cursor()
    cur.execute('SELECT * FROM {} '.format(table[0]))
    rows = cur.fetchall()
        
    conn.close()
    
    nametable = table[0][0]

    return render_template('data.html', rows=rows, nametable = nametable)



In [ ]:
@app.route('/melon3', methods=['POST'])
def melon3():
    ch = request.form['tablename']
    table = []
    table.append([ch])
    nametable = table[0][0]
    return render_template('melon3.html', nametable=nametable)

@app.route('/show1', methods=['POST'])
def show1():
    na = request.form['name']

    ch = request.form['tablename']
    print(ch)
    table = []
    table.append([ch])
    nametable = table[0][0]
    
    conn=sqlite3.connect('music.db', check_same_thread=False)
    conn.row_factory = sqlite3.Row
    # sele = 'select*from {} where singer=?'.format(table[0])
    # cursor.execute(sele,(na,))
    cursor = conn.cursor()
    cursor.execute('''select * from {} where singer=?'''.format(table[0]),(na,))
    row = cursor.fetchall()

    conn.close()

    return render_template('data1.html', rows=row, nametable=nametable)



In [ ]:
@app.route('/melon4', methods=['POST'])
def melon4():
    ch = request.form['tablename']
    table = []
    table.append([ch])
    nametable = table[0][0]
    return render_template('melon4.html', nametable = nametable)

@app.route('/show2', methods=['POST'])
def show2():
    rank = request.form['ranking']
    ch = request.form['tablename']
    print(ch)
    table = []
    table.append([ch])
    nametable = table[0][0]

    conn=sqlite3.connect('music.db', check_same_thread=False)
    conn.row_factory = sqlite3.Row
    cursor = conn.cursor()
    cursor.execute('''select*from {} where ranking=?'''.format(table[0]),(rank,))
    row = cursor.fetchall()

    conn.close()

    return render_template('data2.html', rows=row, nametable = nametable)

In [ ]:
#플라스크 서버 구동
if __name__=='__main__':
    app.run()  #실행되는 컴퓨터 IP 주소, 포트번호 5000